# M-SAWI graphics

This notebook generates the graphic representations of the M-SAWI estimates, as well as determines the best-fitting curve for the data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, root_mean_squared_error

## Initial graphic

In [ ]:
info = pd.read_csv('../data/output/table/accessibility_index/indexes/M-SAWI/M-SAWI_all_territories_norm.txt')
info = info.sort_values('p_(h)',ascending=False)

fig, ax = plt.subplots(figsize=(8, 5))
x = 'p_(h)'

ax.scatter(info[x],info['M-SAWI_EQU'],color='g',label='M-SAWI_EQU',alpha=0.6)
ax.scatter(info[x],info['M-SAWI_PCA'],color='r',label='M-SAWI_PCA',alpha=0.6)
ax.scatter(info[x],info['M-SAWI_AHP'],color='b',label='M-SAWI_AHP',alpha=0.6)

ax.set_xscale('log')
ax.set_xlim(1e4,1e7)
ax.set_ylim(0,0.5)
ax.set_xlabel('log scale Population (hab)',fontsize=14)
ax.set_ylabel('M-SAWI',fontsize=14)
ax.grid()
ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.tight_layout()
plt.savefig('../data/output/image/M-SAWI/M_SAWI_pop.png')

## Best-fitting curve

In [ ]:
def func_exp(x, b, c):
    return np.exp(-b * x ** c)

x_data = np.array(info['p_(h)'])
x_data_log = np.array(np.log(info['p_(h)']))
# M-SAWI
y_data_EQU = np.array(info['M-SAWI_EQU'])
y_data_AHP = np.array(info['M-SAWI_AHP'])
y_data_PCA = np.array(info['M-SAWI_PCA'])
# Fitting
popt_exp_EQU, pcov_exp_EQU = curve_fit(func_exp, x_data_log, y_data_EQU)
popt_exp_AHP, pcov_exp_AHP = curve_fit(func_exp, x_data_log, y_data_AHP)
popt_exp_PCA, pcov_exp_PCA = curve_fit(func_exp, x_data_log, y_data_PCA)
# Errors
r2_exp_EQU = r2_score(y_data_EQU, func_exp(x_data_log, *popt_exp_EQU))
r2_exp_AHP = r2_score(y_data_AHP, func_exp(x_data_log, *popt_exp_AHP))
r2_exp_PCA = r2_score(y_data_PCA, func_exp(x_data_log, *popt_exp_PCA))
rmse_exp_EQU = root_mean_squared_error(y_data_EQU, func_exp(x_data_log, *popt_exp_EQU))
rmse_exp_AHP = root_mean_squared_error(y_data_AHP, func_exp(x_data_log, *popt_exp_AHP))
rmse_exp_PCA = root_mean_squared_error(y_data_PCA, func_exp(x_data_log, *popt_exp_PCA))
# Plot
x_fit = np.linspace(x_data_log.min(), x_data_log.max(), 1000)
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(np.exp(x_fit), func_exp(x_fit, *popt_exp_EQU), label='EQU $R^2$=%7.4f, RMSE=%7.4f\n$b = %7.4f, c = %7.4f$'% (r2_exp_EQU,rmse_exp_EQU,popt_exp_EQU[0],popt_exp_EQU[1]), alpha=0.7, color='g')
ax.plot(np.exp(x_fit), func_exp(x_fit, *popt_exp_PCA), label='PCA $R^2$=%7.4f, RMSE=%7.4f\n$b = %7.4f, c = %7.4f$'% (r2_exp_PCA,rmse_exp_PCA,popt_exp_PCA[0],popt_exp_PCA[1]), alpha=0.7, color='r')
ax.plot(np.exp(x_fit), func_exp(x_fit, *popt_exp_AHP), label='AHP $R^2$=%7.4f, RMSE=%7.4f\n$b = %7.4f, c = %7.4f$'% (r2_exp_AHP,rmse_exp_AHP,popt_exp_AHP[0],popt_exp_AHP[1]), alpha=0.7, color='b')
ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)
textstr = '$Y = exp(-b\cdot X^{c})$'
# Text box properties
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
# Text box position
ax.text(0.05, 0.15, textstr, transform=ax.transAxes, fontsize=18,
        verticalalignment='top', bbox=props)
plt.xlim(1e4,1e7)
plt.ylim(0,0.5)
plt.grid()
plt.xscale('log')
plt.xlabel('Population (log scale)',fontsize=14)
plt.ylabel('M-SAWI',fontsize=14)
plt.tight_layout()
plt.savefig('../data/output/image/M-SAWI/M_SAWI_fitting_curves.png')